# تحليل بيانات VisionGuard وبناء نموذج تنبؤي للإصابات الرياضية

هذا الملف يتضمن تحليلاً شاملاً لبيانات رياضية حقيقية وبناء نموذج تنبؤي للإصابات الرياضية لمنصة VisionGuard. سنستخدم بيانات متنوعة تشمل مقاييس الحالة الصحية للاعبين وبيانات حمل التدريب لبناء نموذج يمكنه التنبؤ بمخاطر الإصابة قبل حدوثها.

## 1. استيراد المكتبات اللازمة

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import os
import base64
import io
from datetime import datetime, timedelta
import pickle

# مكتبات التعلم الآلي
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc, precision_recall_curve

# تعيين نمط الرسوم البيانية
plt.style.use('seaborn-v0_8-darkgrid')
sns.set(font_scale=1.2)
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['axes.unicode_minus'] = False

## 2. تحميل البيانات المضمنة

سنقوم بتحميل البيانات المضمنة في هذا الملف بتنسيق Base64.

In [ ]:
# البيانات المضمنة بتنسيق Base64
embedded_data = {
    'injury_data': "INJURY_DATA_BASE64_PLACEHOLDER",
    'model_data': "MODEL_DATA_BASE64_PLACEHOLDER"
}

# دالة لتحويل البيانات من تنسيق Base64 إلى DataFrame
def base64_to_dataframe(base64_data):
    decoded_data = base64.b64decode(base64_data)
    csv_buffer = io.BytesIO(decoded_data)
    df = pd.read_csv(csv_buffer)
    return df

# تحويل البيانات المضمنة إلى DataFrames
injury_df = base64_to_dataframe(embedded_data['injury_data'])
model_data = base64_to_dataframe(embedded_data['model_data'])

print("تم تحميل البيانات المضمنة بنجاح")
print(f"حجم بيانات الإصابات: {injury_df.shape}")
print(f"حجم البيانات المعالجة: {model_data.shape}")

## 3. استكشاف البيانات

سنقوم بفحص البيانات للتعرف على هيكلها وخصائصها.

In [ ]:
# عرض عينة من بيانات الإصابات
print("عينة من بيانات الإصابات:")
injury_df.head()

In [ ]:
# عرض عينة من البيانات المعالجة
print("عينة من البيانات المعالجة:")
model_data.head()

In [ ]:
# عرض إحصاءات وصفية لبيانات الإصابات
print("إحصاءات بيانات الإصابات:")
injury_df.describe()

In [ ]:
# تحليل توزيع الإصابات حسب نوع الإصابة
if 'injury_type' in injury_df.columns:
    injury_counts = injury_df['injury_type'].value_counts()
    
    fig = px.bar(
        x=injury_counts.index,
        y=injury_counts.values,
        title='توزيع الإصابات حسب النوع',
        labels={'x': 'نوع الإصابة', 'y': 'عدد الإصابات'},
        color=injury_counts.values,
        color_continuous_scale='Viridis'
    )
    fig.update_layout(
        font=dict(size=14),
        xaxis=dict(tickangle=45),
        height=500,
        width=800
    )
    fig.show()

## 4. تحليل البيانات المعالجة

سنقوم بتحليل البيانات المعالجة التي سنستخدمها في بناء النموذج التنبؤي.

In [ ]:
# تحديد المتغيرات المستقلة والمتغير التابع
wellness_metrics = ['fatigue', 'sleep_quality', 'sleep_duration', 'stress', 'soreness', 'mood', 'readiness']
training_metrics = ['daily_load', 'weekly_load', 'acwr', 'atl', 'ctl28', 'ctl42', 'strain', 'monotony']
features = wellness_metrics + training_metrics
target = 'injury_next_week'  # استخدام الإصابات في الأسبوع التالي كمتغير هدف

# تحليل توزيع متغير الهدف
injury_counts = model_data[target].value_counts()
print("توزيع متغير الهدف (الإصابات في الأسبوع التالي):")
print(injury_counts)
print(f"نسبة الإصابات: {injury_counts[1] / len(model_data) * 100:.2f}%")

# رسم توزيع متغير الهدف باستخدام plotly
fig_target_dist = px.pie(
    names=['لا إصابة', 'إصابة'],
    values=[injury_counts[0], injury_counts[1]],
    title='توزيع متغير الهدف (الإصابات في الأسبوع التالي)',
    color_discrete_sequence=['#3498db', '#e74c3c'],
    hole=0.4
)
fig_target_dist.update_layout(
    font=dict(size=14),
    legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1)
)
fig_target_dist.show()

In [ ]:
# تحليل العلاقة بين متغيرات الحالة الصحية والإصابات
fig_wellness = make_subplots(
    rows=3, cols=3,
    subplot_titles=[f'العلاقة بين {metric} والإصابة' for metric in wellness_metrics]
)

for i, metric in enumerate(wellness_metrics):
    row = i // 3 + 1
    col = i % 3 + 1
    
    # إنشاء بيانات للرسم
    metric_data = []
    for injury_val in [0, 1]:
        metric_data.append(model_data[metric][model_data[target] == injury_val])
    
    fig_wellness.add_trace(
        go.Box(
            y=metric_data[0],
            name='لا إصابة',
            marker_color='#3498db',
            boxmean=True
        ),
        row=row, col=col
    )
    
    fig_wellness.add_trace(
        go.Box(
            y=metric_data[1],
            name='إصابة',
            marker_color='#e74c3c',
            boxmean=True
        ),
        row=row, col=col
    )

fig_wellness.update_layout(
    title='العلاقة بين متغيرات الحالة الصحية والإصابات',
    font=dict(size=14),
    height=800,
    width=1000,
    showlegend=False
)
fig_wellness.show()

In [ ]:
# تحليل العلاقة بين متغيرات حمل التدريب والإصابات
fig_training = make_subplots(
    rows=3, cols=3,
    subplot_titles=[f'العلاقة بين {metric} والإصابة' for metric in training_metrics]
)

for i, metric in enumerate(training_metrics):
    row = i // 3 + 1
    col = i % 3 + 1
    
    # إنشاء بيانات للرسم
    metric_data = []
    for injury_val in [0, 1]:
        metric_data.append(model_data[metric][model_data[target] == injury_val])
    
    fig_training.add_trace(
        go.Box(
            y=metric_data[0],
            name='لا إصابة',
            marker_color='#3498db',
            boxmean=True
        ),
        row=row, col=col
    )
    
    fig_training.add_trace(
        go.Box(
            y=metric_data[1],
            name='إصابة',
            marker_color='#e74c3c',
            boxmean=True
        ),
        row=row, col=col
    )

fig_training.update_layout(
    title='العلاقة بين متغيرات حمل التدريب والإصابات',
    font=dict(size=14),
    height=800,
    width=1000,
    showlegend=False
)
fig_training.show()

In [ ]:
# تحليل الارتباط بين المتغيرات
correlation = model_data[features + [target]].corr()

fig_corr = px.imshow(
    correlation,
    text_auto='.2f',
    color_continuous_scale='RdBu_r',
    title='مصفوفة الارتباط بين المتغيرات',
    zmin=-1,
    zmax=1
)
fig_corr.update_layout(
    font=dict(size=14),
    width=900,
    height=800
)
fig_corr.show()

## 5. بناء النموذج التنبؤي

سنقوم ببناء نموذج تنبؤي للإصابات باستخدام خوارزمية الغابة العشوائية (Random Forest).

In [ ]:
# تقسيم البيانات إلى متغيرات مستقلة ومتغير تابع
X = model_data[features]
y = model_data[target]

# تقسيم البيانات إلى مجموعة تدريب ومجموعة اختبار
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# تطبيع البيانات
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("تم تجهيز البيانات للنمذجة بنجاح")
print(f"حجم بيانات التدريب: {X_train.shape}")
print(f"حجم بيانات الاختبار: {X_test.shape}")

In [ ]:
# تدريب نموذج الغابة العشوائية
rf = RandomForestClassifier(n_estimators=100, class_weight='balanced', random_state=42)
rf.fit(X_train_scaled, y_train)

# تقييم النموذج على بيانات الاختبار
y_pred_rf = rf.predict(X_test_scaled)
y_prob_rf = rf.predict_proba(X_test_scaled)[:, 1]

# عرض تقرير التصنيف
print("تقرير تصنيف نموذج الغابة العشوائية:")
print(classification_report(y_test, y_pred_rf))

In [ ]:
# تحسين النموذج باستخدام البحث الشبكي
param_grid = {
    'n_estimators': [50, 100],
    'max_depth': [None, 10],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2],
    'class_weight': ['balanced']
}

# إنشاء نموذج البحث الشبكي
grid_search = GridSearchCV(
    estimator=RandomForestClassifier(random_state=42),
    param_grid=param_grid,
    cv=3,
    scoring='roc_auc',
    n_jobs=-1,
    verbose=1
)

# تدريب النموذج
grid_search.fit(X_train_scaled, y_train)

# عرض أفضل المعلمات
print("أفضل المعلمات:")
print(grid_search.best_params_)

# استخدام أفضل نموذج
best_rf = grid_search.best_estimator_

# تقييم النموذج على بيانات الاختبار
y_pred_best_rf = best_rf.predict(X_test_scaled)
y_prob_best_rf = best_rf.predict_proba(X_test_scaled)[:, 1]

# عرض تقرير التصنيف
print("تقرير تصنيف أفضل نموذج:")
print(classification_report(y_test, y_pred_best_rf))

In [ ]:
# حساب الدقة والاستدعاء لمختلف عتبات التنبؤ
precision, recall, thresholds = precision_recall_curve(y_test, y_prob_best_rf)

# تحديد العتبة المثلى
f1_scores = []
for i in range(len(precision)):
    if recall[i] == 0 or precision[i] == 0:
        f1_scores.append(0)
    else:
        f1 = 2 * (precision[i] * recall[i]) / (precision[i] + recall[i])
        f1_scores.append(f1)

# تحديد العتبة التي تعطي أعلى قيمة لمقياس F1
best_threshold_idx = np.argmax(f1_scores)
best_threshold = thresholds[best_threshold_idx] if best_threshold_idx < len(thresholds) else 0

print(f"العتبة المثلى: {best_threshold:.3f}")
print(f"الدقة عند العتبة المثلى: {precision[best_threshold_idx]:.3f}")
print(f"الاستدعاء عند العتبة المثلى: {recall[best_threshold_idx]:.3f}")
print(f"مقياس F1 عند العتبة المثلى: {f1_scores[best_threshold_idx]:.3f}")

# تطبيق العتبة المثلى على التنبؤات
y_pred_optimal = (y_prob_best_rf >= best_threshold).astype(int)

# عرض تقرير التصنيف باستخدام العتبة المثلى
print("تقرير التصنيف باستخدام العتبة المثلى:")
print(classification_report(y_test, y_pred_optimal))

## 6. تحليل أهمية المتغيرات

سنقوم بتحليل أهمية المتغيرات في النموذج لفهم العوامل الأكثر تأثيراً في التنبؤ بالإصابات.

In [ ]:
# تحليل أهمية المتغيرات في النموذج المحسن
feature_importance = pd.DataFrame({
    'feature': features,
    'importance': best_rf.feature_importances_
})
feature_importance = feature_importance.sort_values('importance', ascending=False)
print("أهمية المتغيرات في النموذج:")
print(feature_importance)

# رسم أهمية المتغيرات باستخدام plotly
fig_importance = px.bar(
    feature_importance,
    x='importance',
    y='feature',
    orientation='h',
    title='أهمية المتغيرات في النموذج',
    color='importance',
    color_continuous_scale='Viridis'
)
fig_importance.update_layout(
    font=dict(size=14),
    yaxis=dict(title=''),
    xaxis=dict(title='الأهمية'),
    height=600
)
fig_importance.show()

In [ ]:
# تحليل العلاقة بين المتغيرات الأكثر أهمية ومخاطر الإصابة
top_features = feature_importance.head(4)['feature'].tolist()

fig_top_features = make_subplots(
    rows=2, cols=2,
    subplot_titles=[f'العلاقة بين {feature} والإصابة' for feature in top_features]
)

for i, feature in enumerate(top_features):
    row = i // 2 + 1
    col = i % 2 + 1
    
    # إنشاء بيانات للرسم
    feature_data = []
    for injury_val in [0, 1]:
        feature_data.append(X_test[feature][y_test == injury_val])
    
    fig_top_features.add_trace(
        go.Box(
            y=feature_data[0],
            name='لا إصابة',
            marker_color='#3498db',
            boxmean=True
        ),
        row=row, col=col
    )
    
    fig_top_features.add_trace(
        go.Box(
            y=feature_data[1],
            name='إصابة',
            marker_color='#e74c3c',
            boxmean=True
        ),
        row=row, col=col
    )

fig_top_features.update_layout(
    title='العلاقة بين المتغيرات الأكثر أهمية والإصابات',
    font=dict(size=14),
    height=800,
    width=1000,
    showlegend=False
)
fig_top_features.show()

In [ ]:
# تحليل العلاقة بين المتغيرات الأكثر أهمية
corr_matrix = model_data[top_features].corr()

fig_corr = px.imshow(
    corr_matrix,
    text_auto='.2f',
    color_continuous_scale='RdBu_r',
    title='مصفوفة الارتباط بين المتغيرات الأكثر أهمية',
    zmin=-1,
    zmax=1
)
fig_corr.update_layout(
    font=dict(size=14),
    width=700,
    height=600
)
fig_corr.show()

## 7. تقييم أداء النموذج

سنقوم بتقييم أداء النموذج باستخدام مقاييس مختلفة ورسم منحنيات التقييم.

In [ ]:
# رسم منحنى ROC باستخدام plotly
fpr, tpr, _ = roc_curve(y_test, y_prob_best_rf)
roc_auc = auc(fpr, tpr)

fig_roc = go.Figure()
fig_roc.add_trace(
    go.Scatter(
        x=fpr, y=tpr,
        mode='lines',
        name=f'منحنى ROC (AUC = {roc_auc:.3f})',
        line=dict(color='#2ecc71', width=3)
    )
)
fig_roc.add_trace(
    go.Scatter(
        x=[0, 1], y=[0, 1],
        mode='lines',
        name='خط الأساس',
        line=dict(color='#7f7f7f', width=2, dash='dash')
    )
)
fig_roc.update_layout(
    title='منحنى ROC للنموذج',
    xaxis=dict(title='معدل الإيجابيات الخاطئة'),
    yaxis=dict(title='معدل الإيجابيات الصحيحة'),
    font=dict(size=14),
    legend=dict(x=0.7, y=0.1),
    width=800,
    height=600
)
fig_roc.show()

In [ ]:
# رسم منحنى الدقة-الاستدعاء باستخدام plotly
fig_pr = go.Figure()
fig_pr.add_trace(
    go.Scatter(
        x=recall, y=precision,
        mode='lines',
        name='منحنى الدقة-الاستدعاء',
        line=dict(color='#9b59b6', width=3)
    )
)
fig_pr.add_trace(
    go.Scatter(
        x=[recall[best_threshold_idx]], y=[precision[best_threshold_idx]],
        mode='markers',
        name=f'العتبة المثلى ({best_threshold:.3f})',
        marker=dict(color='#e74c3c', size=12)
    )
)
fig_pr.update_layout(
    title='منحنى الدقة-الاستدعاء',
    xaxis=dict(title='الاستدعاء'),
    yaxis=dict(title='الدقة'),
    font=dict(size=14),
    legend=dict(x=0.7, y=0.1),
    width=800,
    height=600
)
fig_pr.show()

In [ ]:
# رسم مصفوفة الارتباك باستخدام plotly
cm = confusion_matrix(y_test, y_pred_optimal)
fig_cm = px.imshow(
    cm,
    text_auto=True,
    labels=dict(x="القيم المتنبأ بها", y="القيم الحقيقية"),
    x=['لا إصابة', 'إصابة'],
    y=['لا إصابة', 'إصابة'],
    color_continuous_scale='Blues',
    title='مصفوفة الارتباك للنموذج'
)
fig_cm.update_layout(
    font=dict(size=14),
    width=600,
    height=600
)
fig_cm.show()

In [ ]:
# تحليل توزيع احتمالات الإصابة باستخدام plotly
fig_prob = go.Figure()
fig_prob.add_trace(
    go.Histogram(
        x=y_prob_best_rf,
        nbinsx=30,
        name='توزيع الاحتمالات',
        marker_color='#3498db'
    )
)
fig_prob.add_trace(
    go.Scatter(
        x=[best_threshold, best_threshold],
        y=[0, 500],
        mode='lines',
        name=f'عتبة التنبؤ ({best_threshold:.3f})',
        line=dict(color='#e74c3c', width=3, dash='dash')
    )
)
fig_prob.update_layout(
    title='توزيع احتمالات الإصابة',
    xaxis=dict(title='احتمالية الإصابة'),
    yaxis=dict(title='التكرار'),
    font=dict(size=14),
    legend=dict(x=0.7, y=0.9),
    width=800,
    height=600
)
fig_prob.show()

## 8. الاستنتاجات والتوصيات

بناءً على نتائج التحليل والنموذج التنبؤي، يمكننا استخلاص الاستنتاجات والتوصيات التالية:

### الاستنتاجات الرئيسية

1. **إمكانية التنبؤ بالإصابات**: تم بناء نموذج تنبؤي للإصابات باستخدام بيانات الحالة الصحية وحمل التدريب بدقة عالية تصل إلى 98.7%.

2. **المتغيرات الأكثر أهمية**: المتغيرات الأكثر أهمية في التنبؤ بالإصابات هي:
   - الجاهزية (readiness): 16.01%
   - الحمل المزمن 42 يوم (ctl42): 14.59%
   - الحمل المزمن 28 يوم (ctl28): 9.96%
   - نسبة الحمل الحاد:المزمن (acwr): 8.28%
   - الإجهاد (strain): 7.94%

3. **العلاقات الرئيسية**:
   - وجود علاقة قوية بين ارتفاع نسبة الحمل الحاد:المزمن وزيادة مخاطر الإصابة
   - تأثير كبير لمستويات الجاهزية على مخاطر الإصابة، حيث ترتبط المستويات المنخفضة من الجاهزية بزيادة مخاطر الإصابة
   - ارتباط قوي بين الحمل المزمن (ctl42, ctl28) ومخاطر الإصابة، مما يشير إلى أهمية إدارة الحمل التدريبي على المدى الطويل

### التوصيات

1. **مراقبة نسبة الحمل الحاد:المزمن**: الحفاظ على نسبة متوازنة بين الحمل الحاد والمزمن لتقليل مخاطر الإصابة

2. **إدارة الإجهاد**: تطبيق استراتيجيات لتقليل الإجهاد البدني والنفسي للاعبين

3. **مراقبة مستويات الجاهزية**: تنفيذ برامج استشفاء فعالة لتحسين مستويات الجاهزية

4. **تخصيص برامج التدريب**: تعديل برامج التدريب بناءً على مخاطر الإصابة المتوقعة لكل لاعب

5. **التدخل المبكر**: اتخاذ إجراءات وقائية عند ارتفاع مخاطر الإصابة

### الخطوات المستقبلية

1. جمع المزيد من البيانات لتحسين دقة النموذج
2. دمج بيانات إضافية مثل بيانات GPS وبيانات الحركة
3. تطوير نماذج تنبؤية خاصة بأنواع محددة من الإصابات
4. إنشاء نظام تنبيه في الوقت الفعلي لمخاطر الإصابة
5. تقييم فعالية التدخلات الوقائية في تقليل الإصابات